### Name: Kevin J. Schwint
#### Date: April 18, 2023

# Project #3:

# Web Scraped Data Analysis

# Web Scraping with requests and Beautiful Soup

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import matplotlib.pyplot as plt
# from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
# Define the URL to scrape
url = 'https://www.hockey-reference.com/teams/NJD/index.html'
# Send a GET request to the URL
response = requests.get(url)
# Parse the HTML content
soup = bs(response.content, 'html.parser')

In [4]:
print(soup.prettify())
type(soup)

<!DOCTYPE html>
<html class="no-js" data-root="/home/hr/build" data-version="klecko-" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
   <link href="https://cdn.ssref.net/req/202304261" rel="dns-prefetch"/>
   <!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
   <script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME

bs4.BeautifulSoup

### Scrape data from one page and store in a DataFrame

In [5]:
player_list = [ ]
stats_table = soup.find(class_='stats_table').tbody
for row in stats_table.find_all('tr'):
    player = row.find(attrs={'data-stat':'player','scope':'row'}).text
    print(player)
    pos= row.find(attrs={'data-stat':'position'}).text
    player_list.append({'player':player,'position':pos})
#player_list

player_df = pd.DataFrame(player_list)

Jonathan Bernier


AttributeError: 'NoneType' object has no attribute 'text'

In [6]:
# Find the roster table
roster_table = soup.find('table', {'id': 'roster'})

# Extract the roster data
roster_data = []
if roster_table:
    for row in roster_table.find_all('tr'):
        cells = row.find_all('td')
        if cells:
            player_name = cells[0].text.strip()
            player_country = cells[1].text.strip()
            player_position = cells[2].text.strip()
            player_age = cells[3].text.strip()
            player_salary = cells[10].text.strip()
            roster_data.append([player_name, player_country, player_position, player_age, player_salary])

# Print the roster data
if roster_data:
    print("Team Roster:")
    print("{:<30} {:<10} {:<10} {:<10} {:<10}".format("Player Name", "Country", "Position", "Age", "Salary"))
    for player in roster_data:
        print("{:<30} {:<10} {:<10} {:<10} {:<10}".format(player[0], player[1], player[2], player[3], player[4]))
else:
    print("No roster data found.")
    
roster_df = pd.DataFrame(roster_data)

Team Roster:
Player Name                    Country    Position   Age        Salary    
Kevin Bahl                     ca         D          22         $750,000  
Nathan Bastian                 ca         F          25         $875,000  
Mackenzie Blackwood            ca         G          26         $4,125,000
Jesper Boqvist                 se         F          24         $874,125  
Jesper Bratt                   se         F          24         $5,450,000
Ryan Graves                    ca         D          27         $4,000,000
Dougie Hamilton                ca         D          29         $3,300,000
Erik Haula                     fi         F          31         $2,500,000
Nico Hischier (C)              ch         F          24         $4,500,000
Jack Hughes                    us         F          21         $7,000,000
Luke Hughes                    us         D          19         $832,500  
Curtis Lazar                   ca         F          27         $1,000,000
John Marino 

In [7]:
roster_df

,0,1,2,3,4
0,Kevin Bahl,ca,D,22,"$750,000"
1,Nathan Bastian,ca,F,25,"$875,000"
2,Mackenzie Blackwood,ca,G,26,"$4,125,000"
3,Jesper Boqvist,se,F,24,"$874,125"
4,Jesper Bratt,se,F,24,"$5,450,000"
5,Ryan Graves,ca,D,27,"$4,000,000"
6,Dougie Hamilton,ca,D,29,"$3,300,000"
7,Erik Haula,fi,F,31,"$2,500,000"
8,Nico Hischier (C),ch,F,24,"$4,500,000"
9,Jack Hughes,us,F,21,"$7,000,000"


### Create a function to extract data from one web page so it can be used to extract from multiple pages.

In [10]:
import requests
from bs4 import BeautifulSoup

def collect_player_information(team_name, year):

    # Create an empty dictionary to store the collected player information
    player_info = {}

    # Construct the URL for the team's page for the required year
    url = f"https://www.hockey-reference.com/teams/{team_name}/{year}.html#all_roster"
#   url = f" https://www.hockey-reference.com/teams/NJD/2019.html"

    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content of the response
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract player information from the parsed HTML
    player_names = soup.find_all('td', {'data-stat': 'player'})
    if player_names:
        # Create a list to store player names for the current year
        year_player_names = []
        for player_name in player_names:
            year_player_names.append(player_name.text.strip())
        # Add the year_player_names list to the player_info dictionary
        player_info[str(year)] = year_player_names

    # Extract the roster data
    roster_table = soup.find('table', {'id': 'roster'})
    roster_data = []
    if roster_table:
        for row in roster_table.find_all('tr'):
            cells = row.find_all('td')
            if cells:
                year_player_name = cells[0].text.strip()
                year_player_country = cells[1].text.strip()
                year_player_position = cells[2].text.strip()
                year_player_age = cells[3].text.strip()
                year_player_salary = cells[10].text.strip()
                # Append the year_player_info to the roster_data list
                year_player_info = [year_player_name, year_player_country, year_player_position, year_player_age, year_player_salary]
                roster_data.append(year_player_info)

                columns = row.find_all('td', {'data-stat': lambda x: x in ['player', 'pos', 'age', 'games_played', 'goals', 'assists', 'points', 'plus_minus', 'pen_min', 'goals_ev', 'goals_pp', 'goals_sh', 'goals_gw', 'goals_ev']})
                if columns:
                    player_name = columns[0].text.strip()
                    player_position = columns[1].text.strip()
                    player_age = columns[2].text.strip()
                    player_gp = columns[3].text.strip()
                    player_g = columns[4].text.strip()
                    player_a = columns[5].text.strip()
                    player_pts = columns[6].text.strip()
                    player_pm = columns[7].text.strip()
                    player_pim = columns[8].text.strip()
                    player_ev = columns[9].text.strip()
                    player_pp = columns[10].text.strip()
                    player_sh = columns[11].text.strip()
                    player_gw = columns[12].text.strip()
#                   player_ev = columns[13].text.strip()

                    player_info[player_name] = {
                        'Position': player_position,
                        'Age': player_age,
                        'GP': player_gp,
                        'G': player_g,
                        'A': player_a,
                        'PTS': player_pts,
                        'PM': player_pm,
                        'PIM': player_pim,
                        'EV': player_ev,
                        'PP': player_pp,
                        'SH': player_sh,
                        'GW': player_gw,
#                       'EV': player_ev
                    }

    return player_info

# New Jersey Devils - 2019
team_name = 'NJD'
year = 2019
player_info = collect_player_information(team_name, year)
print(player_info)

NameError: name 'year_player_salary' is not defined

### Can't seem to get this one above to work.

## Collecting Player Information from Multiple Pages

You will need to create a web scraping pipeline to do this.  Remember the main steps:
1. Gather links -- develop a strategy for scraping past posts
2. Scrape the same data from each page
3. Clean the data as necessary

Save the information as one dictionary for each post and put all the posts in a list called `pipeline_list`. 

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def collect_player_information_df(team_name, year):
    # Create an empty DataFrame to store the collected player information
    df = pd.DataFrame(columns=['Year', 'Player Name', 'Position', 'Age', 'GP', 'G', 'A', 'PTS', 'PM', 'PIM', 'EV', 'PP', 'SH', 'GW'])

    # Construct the URL for the team's page for the required year
    url = f"https://www.hockey-reference.com/teams/{team_name}/{year}.html"

    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content of the response
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract player information from the parsed HTML
    player_rows = soup.find_all('tr', {'class': 'full_table'})
    for row in player_rows:
        # Extract player name and position
        player_name = row.find('td', {'data-stat': 'player'}).text.strip()
        player_position = row.find('td', {'data-stat': 'pos'}).text.strip()

        # Extract player age
        try:
            player_age = int(row.find('td', {'data-stat': 'age'}).text.strip())
        except ValueError:
            player_age = None

        # Extract player statistics
        player_stats = row.find_all('td', {'class': 'right'})
        player_gp = int(player_stats[0].text.strip())
        player_g = int(player_stats[1].text.strip())
        player_a = int(player_stats[2].text.strip())
        player_pts = int(player_stats[3].text.strip())
        player_pm = int(player_stats[4].text.strip())
        player_pim = int(player_stats[5].text.strip())
        player_ev = int(player_stats[6].text.strip())
        player_pp = int(player_stats[7].text.strip())
        player_sh = int(player_stats[8].text.strip())
        player_gw = int(player_stats[9].text.strip())

        # Create a dictionary with player information
        player_info = {
            'Year': year,
            'Player Name': player_name,
            'Position': player_position,
            'Age': player_age,
            'GP': player_gp,
            'G': player_g,
            'A': player_a,
            'PTS': player_pts,
            'PM': player_pm,
            'PIM': player_pim,
            'EV': player_ev,
            'PP': player_pp,
            'SH': player_sh,
            'GW': player_gw
        }

        # Append the player_info dictionary to the main df
        df = df.append(player_info, ignore_index=True)

    return df

# New Jersey Devils - Last 5 Years
team_name = 'NJD'
player_info_df = pd.DataFrame(columns=['Year', 'Player Name', 'Position', 'Age', 'GP', 'G', 'A', 'PTS', 'PM', 'PIM', 'EV', 'PP', 'SH', 'GW'])
for year in range(2018, 2023):
    year_df = collect_player_information_df(team_name, year)
    player_info_df = pd.concat([player_info_df, year_df])

print(player_info_df)


Empty DataFrame
Columns: [Year, Player Name, Position, Age, GP, G, A, PTS, PM, PIM, EV, PP, SH, GW]
Index: []


In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def collect_player_names_df(team_name, start_year, end_year):
    """
    Collects player information from multiple web pages for a given team and multiple years.

    Args:
        team_name (str): The name of the team.
        start_year (int): The start year for collecting player information.
        end_year (int): The end year for collecting player information.

    Returns:
        pd.DataFrame: A Pandas DataFrame containing the collected player information.
    """
# Create an empty DataFrame to store the collected player information
    df = pd.DataFrame(columns=['Year', 'Player Name'])

# Loop through the years from start_year to end_year
    for year in range(start_year, end_year + 1):
# Construct the URL for the team's page for the current year
        url = f"https://www.hockey-reference.com/teams/{team_name}/{year}.html"

# Send a GET request to the URL
        response = requests.get(url)

# Parse the HTML content of the response
        soup = BeautifulSoup(response.content, 'html.parser')

# Extract player information from the parsed HTML
# Example: Extract player names
        player_names = soup.find_all('td', {'data-stat': 'player'})
        if player_names:
# Create a list to store player names for the current year
            year_player_names = []
            for player_name in player_names:
                year_player_names.append(player_name.text.strip())
# Create a DataFrame with the year_player_names list and the current year as columns
            year_df = pd.DataFrame({'Year': [year] * len(year_player_names), 'Player Name': year_player_names})
            # Append the year_df to the main df
            df = df.append(year_df, ignore_index=True)

# You can add more data extraction logic here based on your specific requirements

    return df

# New Jersey Develis - Last 5 Years
team_name = 'NJD'
start_year = 2019
end_year = 2023
player_info_df = collect_player_names_df(team_name, start_year, end_year)
print(player_info_df)

C:\Users\kjs1\AppData\Local\Temp\28\ipykernel_49120\2488825598.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(year_df, ignore_index=True)


     Year          Player Name
0    2019       Kenny Agostino
1    2019        Joey Anderson
2    2019       Nathan Bastian
3    2019  Mackenzie Blackwood
4    2019          Brian Boyle
..    ...                  ...
554  2023          Ryan Graves
555  2023           Timo Meier
556  2023          Tomáš Tatar
557  2023        Tyce Thompson
558  2023   Yegor Sharangovich

[559 rows x 2 columns]


In [22]:
player_info_df

,Year,Player Name
0,2019,Kenny Agostino
1,2019,Joey Anderson
2,2019,Nathan Bastian
3,2019,Mackenzie Blackwood
4,2019,Brian Boyle
...,...,...
554,2023,Ryan Graves
555,2023,Timo Meier
556,2023,Tomáš Tatar
557,2023,Tyce Thompson


In [23]:
import requests
from bs4 import BeautifulSoup

def collect_player_information(team_name, start_year, end_year):
    """
    Collects player information from multiple web pages for a given team and multiple years.

    Args:
        team_name (str): The name of the team.
        start_year (int): The start year for collecting player information.
        end_year (int): The end year for collecting player information.

    Returns:
        list: A list of dictionaries containing the collected player information for each year.
    """
    # Create an empty list to store the collected player information for each year
    pipeline_list = []

    # Loop through the years from start_year to end_year
    for year in range(start_year, end_year + 1):
        # Construct the URL for the team's page for the current year
        url = f"https://www.hockey-reference.com/teams/{team_name}/{year}.html"

        # Send a GET request to the URL
        response = requests.get(url)

        # Parse the HTML content of the response
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract player information from the parsed HTML
        # Example: Extract player names
        player_names = soup.find_all('td', {'data-stat': 'player'})
        if player_names:
            # Create a dictionary to store player information for the current year
            year_player_info = {'Year': year, 'Player Names': []}
            for player_name in player_names:
                year_player_info['Player Names'].append(player_name.text.strip())
            # Append the year_player_info dictionary to the pipeline_list
            pipeline_list.append(year_player_info)

        # You can add more data extraction logic here based on your specific requirements

    return pipeline_list

# Example usage
team_name = 'NJD'
start_year = 2019
end_year = 2023
player_info_pipeline = collect_player_information(team_name, start_year, end_year)
print(player_info_pipeline)

[{'Year': 2019, 'Player Names': ['Kenny Agostino', 'Joey Anderson', 'Nathan Bastian', 'Mackenzie Blackwood', 'Brian Boyle', 'Jesper Bratt', 'Will Butcher', 'Connor Carrick', 'Blake Coleman', 'Jean-Sebastien Dea', 'Kurtis Gabriel', 'Brandon Gignac', 'Andy Greene\xa0(C)', 'Eric Gryba', 'Taylor Hall', 'Nico Hischier', 'Joshua Jacobs', 'Marcus Johansson', 'Keith Kinkaid', 'Nick Lappin', 'Ben Lovejoy', 'Michael McLeod', 'Mirco Müller', 'Ryan Murphy', 'Stefan Noesen', 'Kyle Palmieri', 'Blake Pietila', 'John Quenneville', 'Kevin Rooney', 'Steven Santini', 'Cory Schneider', 'Brett Seney', 'Damon Severson', 'Drew Stafford', 'Eric Tangradi', 'Sami Vatanen', 'Colton White', 'Miles Wood', 'Egor Yakovlev', 'Pavel Zacha', 'Travis Zajac', 'Kyle Palmieri', 'Nico Hischier', 'Travis Zajac', 'Damon Severson', 'Taylor Hall', 'Blake Coleman', 'Jesper Bratt', 'Will Butcher', 'Marcus Johansson', 'Pavel Zacha', 'Andy Greene', 'Miles Wood', 'Brian Boyle', 'Sami Vatanen', 'Brett Seney', 'Drew Stafford', 'Kenny 

### Player Information for a specified Team and a specified Year

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def collect_player_information(team_name, year):
    """
    Collects player information from one web page for a given team and a given year.

    Args:
        team_name (str): The name of the team.
        year (int): The year for collecting player information.

    Returns:
        dict: A dictionary containing the collected player information.
    """

    # Create an empty dictionary to store the collected player information
    player_info = {}

    # Construct the URL for the team's page for the required year
    url = f"https://www.hockey-reference.com/teams/{team_name}/{year}.html"

    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content of the response
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract player information from the parsed HTML
    player_names = soup.find_all('td', {'data-stat': 'player'})
    if player_names:
        # Create a list to store player names for the current year
        year_player_names = []
        for player_name in player_names:
            year_player_names.append(player_name.text.strip())

        # Add the year_player_names list to the player_info dictionary
        player_info[str(year)] = year_player_names

    roster_table = soup.find('table', {'id': 'roster'})
    roster_data = []
    if roster_table:
        for row in roster_table.find_all('tr'):
            cells = row.find_all('td')
            if cells:
                year_player_name = cells[0].text.strip()
                year_player_country = cells[1].text.strip()
                year_player_position = cells[2].text.strip()
                year_player_age = cells[3].text.strip()
#               year_player_salary = cells[10].text.strip()

                player_info = {
                    'Name': year_player_name,
                    'Country': year_player_country,
                    'Position': year_player_position,
                    'Age': year_player_age,
#                   'Salary': year_player_salary,
                }
                roster_data.append(player_info)

    # Print the roster data
    if roster_data:
        print("Team Roster:")
        print("{:<30} {:<10} {:<10} {:<10} {:<10}".format("Player Name", "Country", "Position", "Age", "Salary"))
        for player in roster_data:
#           print("{:<30} {:<10} {:<10} {:<10} {:<10}".format(player['Name'], player['Country'], player['Position'], player['Age'], player['Salary']))
            print("{:<30} {:<10} {:<10} {:<10} {:<10}".format(player['Name'], player['Country'], player['Position'], player['Age']))
    else:
        print("No roster data found.")

    # Create a pandas DataFrame from the roster data
    roster_df = pd.DataFrame(roster_data)

    return player

# New Jersey Develis - Last 5 Years
team_name = 'NJD'
year = 2019
player_info_df = collect_player_information(team_name, year)
print(player_info)

Team Roster:
Player Name                    Country    Position   Age        Salary    


IndexError: Replacement index 4 out of range for positional args tuple

### Also having trouble wit this one above.

### Player Statistics for a specified Team and a specified Year

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_player_data(url):
    """
    Extract player data from a given hockey-reference.com URL for a specific year.
    Returns a Pandas DataFrame with player information.
    """
    # Send a GET request to the URL
    response = requests.get(url)

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing player stats
    table = soup.find('table', {'id': 'skaters'})

    # Extract the column headers
    headers = [th.text for th in table.find_all('th')]

    # Extract the player rows
    rows = []
    for tr in table.find_all('tr')[1:]:
        # Extract the player stats from each row
        data = [td.text for td in tr.find_all('td')]
        row = dict(zip(headers, data))
        rows.append(row)

    # Create a Pandas DataFrame from the rows
    df = pd.DataFrame(rows)

    return df

# Example usage
base_url = 'https://www.hockey-reference.com/teams/NJD/'
years = ['2019', '2020', '2021', '2022', '2023']
pipeline_list = []

for year in years:
    url = base_url + year + '.html'
    df = extract_player_data(url)
    pipeline_list.append(df)
    
# Concatenate the dataframes into one
all_data = pd.concat(pipeline_list, ignore_index=True)

# Print the resulting dataframe
print(all_data)

         Scoring Goals Assists Ice Time Point Shares   Rk Player   Age    Pos  \
0    NaN     NaN   NaN     NaN      NaN          NaN  NaN    NaN   NaN    NaN   
1      0      RW    23      50       16           11  224   12.1  1343  18:09   
2      0       C    30      47       14            3  160   10.6  1249  18:06   
3      0       C    27      46       12            5  120   15.8  1542  19:16   
4      1       D    28      39        8            3  146    7.5  1808  22:03   
..   ...     ...   ...     ...      ...          ...  ...    ...   ...    ...   
190    0      28    LW       2        0            0    0      0     0      1   
191    0      27     C       4        0           -1    0      0     0      1   
192    0      22     G      18        0            0    0      0     0      0   
193    0      23    RW       2        0           -1    0      0     0      1   
194   50                    82      769          277  376     94    10   2821   

       GP    G      A   PTS

### Plot Bar Chart of Team Goals for each Year

In [ ]:
sns.histplot(data=df, x="age", hue="gender", multiple="dodge", binwidth=1, shrink=0.9,
            hue_order=["Man","Woman","Prefer not to say"],
            bins=["18 and younger","19-20","21-24","25-34","35-64"])
plt.title("Age is Mainly Under 20")
plt.xlabel("Age")
plt.ylabel("Number of Students")
# plt.legend(title="Gender")

### Plot Bar Chart of Highest Player Salary for each of last 5 Years

### Plot Bar Chart of Average Team Salaries for each of last 5 Years

### Determine Most Improved Player in Terms of Goals Scored over of last 5 Years

### Plot  Bar Chart of Most Common Player Nationality for each of last 5 Years